<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/1_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
pi = math.pi

np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# MODEL
inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(64, activation='sigmoid', name='hidden_1')(inputs)
hidden_2 = layers.Dense(64, activation='sigmoid', name='hidden_2')(hidden_1)
hidden_3 = layers.Dense(64, activation='sigmoid', name='hidden_3')(hidden_2)
outputs = layers.Dense(1, activation='linear', name="fx")(hidden_3)
model = keras.Model(inputs=inputs, outputs=outputs, name='poisson')
model.summary()

Model: "poisson"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 64)                192       
_________________________________________________________________
hidden_2 (Dense)             (None, 64)                4160      
_________________________________________________________________
hidden_3 (Dense)             (None, 64)                4160      
_________________________________________________________________
fx (Dense)                   (None, 1)                 65        
Total params: 8,577
Trainable params: 8,577
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# DATA
import itertools

n = 30                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P_in, P_border = [],[]
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    if(i == 0 or i == n-1 or j == 0 or j == n-1):
        P_border.append(p)
    else:
        P_in.append(p)

# upsampling P_border that its size is equal to P_in
for i in range(len(P_in)-len(P_border)):
    j = i%(len(P_border))
    P_border.append(P_border[j])

# set batch_size and shuffle_size
batch_size = int(len(P_border)/392)
bs = int(len(P_border))

# convert numpy array, P_in and P_border to Tensorflow Dataset
P_in = tf.convert_to_tensor(P_in, dtype=tf.float32)
P_border = tf.convert_to_tensor(P_border, dtype=tf.float32)

P_in = tf.data.Dataset.from_tensor_slices((P_in))
P_in = P_in.shuffle(buffer_size=bs).batch(batch_size)

P_border = tf.data.Dataset.from_tensor_slices((P_border))
P_border = P_border.shuffle(buffer_size=bs).batch(batch_size)

In [ ]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
# TRAIN FUNCTION
@tf.function
def train_step(P_in, P_border):
    with tf.GradientTape(persistent=True) as tape:
        loss_in, loss_border = [0], [0]
        # interior loss
        for i in range(P_in.shape[0]):
            v = tf.reshape(P_in[i], shape=(1,2))
            tape.watch(v)
            model_vals = model(v)

            # calculating u_xx and u_yy
            u_x_y = tape.gradient(model_vals, v)
            u_xx = tape.gradient(u_x_y[0][0], v)[0][0]
            u_yy = tape.gradient(u_x_y[0][1], v)[0][1]

            # calculating f(x,y)
            f = -2*pi*pi* tf.math.sin(pi*v[0][0]) * tf.math.sin(pi*v[0][1])

            # calculating loss_in
            loss_in += tf.math.square(u_xx + u_yy - f)

        # boundary loss
        for j in range(P_border.shape[0]):
            v = tf.reshape(P_border[j], shape=(1,2))
            loss_border += tf.math.square(model(v))
        
        # loss function
        batch_loss = loss_in + loss_border
    
    # update weights
    grads = tape.gradient(batch_loss, model.trainable_weights)          # gradient of loss function with respect to w
    optimizer.apply_gradients(zip(grads, model.trainable_weights))      # update w

    return batch_loss

In [ ]:
# TRAIN
epochs = 1000
losses = []
start_time = time.time()
for epoch in range(epochs+1):

    optimizer.learning_rate.assign(1/(10**(int(epoch/400) + 2)))

    loss = 0
    for (P_in_batch,P_border_batch) in itertools.zip_longest(P_in,P_border):
        batch_loss = train_step(P_in_batch, P_border_batch)
        loss += batch_loss
    losses.append(loss.numpy())
    if(epoch%100 == 0):
        print("[%4s] Training loss: %10f \t learning_rate: %f" % (epoch, float(loss), optimizer.learning_rate.numpy()))

print("Total time: %.2f minutes" % ((time.time() - start_time)/60))

[   0] Training loss: 25832.238281 	 learning_rate: 0.010000
[ 100] Training loss: 138.525681 	 learning_rate: 0.010000
[ 200] Training loss: 239.468384 	 learning_rate: 0.010000
[ 300] Training loss:  21.017834 	 learning_rate: 0.010000
[ 400] Training loss:   3.231767 	 learning_rate: 0.001000
[ 500] Training loss:   2.594587 	 learning_rate: 0.001000
[ 600] Training loss:   2.575521 	 learning_rate: 0.001000
[ 700] Training loss:   1.943766 	 learning_rate: 0.001000
[ 800] Training loss:   0.795060 	 learning_rate: 0.000100
[ 900] Training loss:   0.765025 	 learning_rate: 0.000100
[1000] Training loss:   0.738349 	 learning_rate: 0.000100
Total time: 9.20 minutes


In [ ]:
z = tf.convert_to_tensor([[0.0, 0.0]])
print("f(0.0, 0.0) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.1, 0.1]])
print("f(0.1, 0.1) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.2, 0.2]])
print("f(0.2, 0.2) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.3, 0.3]])
print("f(0.3, 0.3) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.4, 0.4]])
print("f(0.4, 0.4) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.5, 0.5]])
print("f(0.5, 0.5) \t = %.2f" % model(z).numpy()[0][0])

f(0.0, 0.0) 	 = -0.02
f(0.1, 0.1) 	 = 0.09
f(0.2, 0.2) 	 = 0.35
f(0.3, 0.3) 	 = 0.66
f(0.4, 0.4) 	 = 0.91
f(0.5, 0.5) 	 = 1.01


In [ ]:
model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson/assets


In [ ]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

z = tf.convert_to_tensor([[0.5, 0.5]])
print("f(0.5, 0.5) \t = %.2f" % model1(z).numpy()[0][0])

f(0.5, 0.5) 	 = 1.01
